# GA Customer Revenue Prediction

this jupyter notebook will be using fastai lesson 1 as a guide to create a random forest prediction model for the kaggle competition Google analytics customer revenue prediction

first lets's import everything

to get this set up:

git clone https://github.com/fastai/fastai  
cd fastai  
conda create -n fastai python=3.6 anaconda  
conda env update  
source activate fastai  

In [1]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

/home/connell/mlSrc/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


go get the data you'll need an account https://www.kaggle.com/c/ga-customer-revenue-prediction/data

mkdir data
cd data
mkdir analytics << put your data here root should be the same as this notebook

In [2]:
PATH = "data/analytics/"

In [3]:
!ls {PATH}

test.csv  train.csv


In [4]:
df_raw = pd.read_csv(f'{PATH}train.csv', low_memory=False, 
                     parseb_dates=["date", "visitStartTime"])

In [35]:
len(df_raw.index)

903653

In [ ]:
df_train = 

lets make sure our times got imported correctly

In [5]:
df_raw['visitStartTime'].dtype

dtype('O')

uh oh let's convert that to a datetime that pandas can parse

In [6]:
df_raw['visitStartTime'] = pd.to_datetime(df_raw['visitStartTime'],unit='s')

Let's add our datepart data so we have columns defining beginning of month time of day etc. we could add holidays as an added bonus

In [7]:
add_datepart(df_raw, 'visitStartTime')
add_datepart(df_raw, 'date')

ok so we have the data let's take a look


In [8]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [9]:
display_all(df_raw.tail().T)

,903648,903649,903650,903651,903652
channelGrouping,Social,Social,Social,Social,Social
device,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...","{""browser"": ""Chrome"", ""browserVersion"": ""not a...","{""browser"": ""Android Webview"", ""browserVersion...","{""browser"": ""Chrome"", ""browserVersion"": ""not a...","{""browser"": ""Chrome"", ""browserVersion"": ""not a..."
fullVisitorId,5123779100307500332,7231728964973959842,5744576632396406899,2709355455991750775,0814900163617805053
geoNetwork,"{""continent"": ""Americas"", ""subContinent"": ""Car...","{""continent"": ""Asia"", ""subContinent"": ""Souther...","{""continent"": ""Asia"", ""subContinent"": ""Eastern...","{""continent"": ""Asia"", ""subContinent"": ""Southea...","{""continent"": ""Americas"", ""subContinent"": ""Cen..."
sessionId,5123779100307500332_1483554750,7231728964973959842_1483543798,5744576632396406899_1483526434,2709355455991750775_1483592857,0814900163617805053_1483574474
socialEngagementType,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged
totals,"{""visits"": ""1"", ""hits"": ""17"", ""pageviews"": ""15...","{""visits"": ""1"", ""hits"": ""18"", ""pageviews"": ""13...","{""visits"": ""1"", ""hits"": ""24"", ""pageviews"": ""21...","{""visits"": ""1"", ""hits"": ""24"", ""pageviews"": ""22...","{""visits"": ""1"", ""hits"": ""31"", ""pageviews"": ""31..."
trafficSource,"{""referralPath"": ""/yt/about/"", ""campaign"": ""(n...","{""referralPath"": ""/yt/about/"", ""campaign"": ""(n...","{""referralPath"": ""/yt/about/ko/"", ""campaign"": ...","{""referralPath"": ""/l.php"", ""campaign"": ""(not s...","{""referralPath"": ""/yt/about/es-419/"", ""campaig..."
visitId,1483554750,1483543798,1483526434,1483592857,1483574474
visitNumber,1,1,1,1,1


looks like we have quite a few dictionaries lets take a look at describe to see our averages then figure out how to  handle those

In [10]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
channelGrouping,903653,8,Organic Search,381561,NaN,NaN,NaN,NaN,NaN,NaN,NaN
device,903653,176,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",284522,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fullVisitorId,903653,714167,1957458976293878100,278,NaN,NaN,NaN,NaN,NaN,NaN,NaN
geoNetwork,903653,39032,"{""continent"": ""Americas"", ""subContinent"": ""Nor...",65912,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sessionId,903653,902755,4608007153632839799_1474872716,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
socialEngagementType,903653,1,Not Socially Engaged,903653,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals,903653,14840,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...",362071,NaN,NaN,NaN,NaN,NaN,NaN,NaN
trafficSource,903653,26061,"{""campaign"": ""(not set)"", ""source"": ""google"", ...",297390,NaN,NaN,NaN,NaN,NaN,NaN,NaN
visitId,903653,NaN,NaN,NaN,1.48501e+09,9.02212e+06,1.47003e+09,1.47756e+09,1.48395e+09,1.49276e+09,1.50166e+09
visitNumber,903653,NaN,NaN,NaN,2.2649,9.28373,1,1,1,1,395


I notice a couple things from this data first off, there's about 6 thousand instances of people having the exact same visitStartTime 🤔 

the only two continuos varibles are visitId and visitNumber

there are quite a few dictionaries, I'm guessing we should extrapolate each key to its own new column

I'm out of coffee

so let's try splitting these dicts up into new columns with pd.series

In [11]:
import json
def apply_json(df, col):
    try:
        df[col] = df[col].apply(json.loads)
        return df[col]
    except:
        return df[col]

In [12]:
df_raw['geoNetwork'] = apply_json(df_raw, 'geoNetwork')
df_raw['device'] = apply_json(df_raw, 'device')
df_raw['totals'] = apply_json(df_raw, 'totals')
df_raw['trafficSource'] = apply_json(df_raw, 'trafficSource')

In [13]:
df_raw = df_raw.drop('device', 1).assign(**pd.DataFrame(df_raw.device.values.tolist()))
df_raw = df_raw.drop('geoNetwork', 1).assign(**pd.DataFrame(df_raw.geoNetwork.values.tolist()))
df_raw = df_raw.drop('totals', 1).assign(**pd.DataFrame(df_raw.totals.values.tolist()))
df_raw = df_raw.drop('trafficSource', 1).assign(**pd.DataFrame(df_raw.trafficSource.values.tolist()))
df_raw = df_raw.drop('adwordsClickInfo', 1).assign(**pd.DataFrame(df_raw.adwordsClickInfo.values.tolist()))

In [14]:
display_all(df_raw.tail().T)

,903648,903649,903650,903651,903652
channelGrouping,Social,Social,Social,Social,Social
fullVisitorId,5123779100307500332,7231728964973959842,5744576632396406899,2709355455991750775,0814900163617805053
sessionId,5123779100307500332_1483554750,7231728964973959842_1483543798,5744576632396406899_1483526434,2709355455991750775_1483592857,0814900163617805053_1483574474
socialEngagementType,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged,Not Socially Engaged
visitId,1483554750,1483543798,1483526434,1483592857,1483574474
visitNumber,1,1,1,1,1
visitStartTimeYear,2017,2017,2017,2017,2017
visitStartTimeMonth,1,1,1,1,1
visitStartTimeWeek,1,1,1,1,1
visitStartTimeDay,4,4,4,5,5


In [15]:
display_all(df_raw['transactionRevenue'].describe(include='all').T)

count        11515
unique        5332
top       16990000
freq           256
Name: transactionRevenue, dtype: object

In [16]:
df_raw['transactionRevenue'].fillna(0, inplace=True)

In [17]:
df_raw['transactionRevenue'] = df_raw['transactionRevenue'].astype(np.int64)

In [18]:
df_raw['transactionRevenue'] = np.log(df_raw['transactionRevenue'], where=(df_raw['transactionRevenue']!=0))

In [19]:
display_all(df_raw['transactionRevenue'].describe(include='all').T)

count    903653.000000
mean          0.227118
std           2.003710
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          23.864375
Name: transactionRevenue, dtype: float64

ok let's write a function to check the if the datatype is a list, split it into new columns and drop the prior from the dataframe, optionally we should not creat a new column if all the values are the same. I have a feeling theres a function in the fastai library that alread does this

let's drop all these not available in demo dataset columns

In [21]:
df_raw = df_raw.drop('targetingCriteria', 1)

I should probably do that is null thing jeremy was talking about but you know what they say, the Muad'Dib understand the circumstance (I'm reading dune)

In [22]:
train_cats(df_raw)

train categories and remove all the columns that have 0 uniques


In [23]:
cols = list(df_raw)
nunique = df_raw.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df_raw.drop(cols_to_drop, axis=1)

,channelGrouping,fullVisitorId,sessionId,visitId,visitNumber,visitStartTimeYear,visitStartTimeMonth,visitStartTimeWeek,visitStartTimeDay,visitStartTimeDayofweek,...,adContent,campaign,keyword,medium,referralPath,source,adNetworkType,gclId,page,slot
0,Organic Search,1131660440785968503,1131660440785968503_1472830385,1472830385,1,2016,9,35,2,4,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN
1,Organic Search,377306020877927890,377306020877927890_1472880147,1472880147,1,2016,9,35,3,5,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN
2,Organic Search,3895546263509774583,3895546263509774583_1472865386,1472865386,1,2016,9,35,3,5,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN
3,Organic Search,4763447161404445595,4763447161404445595_1472881213,1472881213,1,2016,9,35,3,5,...,NaN,(not set),google + online,organic,NaN,google,NaN,NaN,NaN,NaN
4,Organic Search,27294437909732085,27294437909732085_1472822600,1472822600,2,2016,9,35,2,4,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN
5,Organic Search,2938943183656635653,2938943183656635653_1472807194,1472807194,1,2016,9,35,2,4,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN
6,Organic Search,1905672039242460897,1905672039242460897_1472817241,1472817241,1,2016,9,35,2,4,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN
7,Organic Search,537222803633850821,537222803633850821_1472812602,1472812602,1,2016,9,35,2,4,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN
8,Organic Search,4445454811831400414,4445454811831400414_1472805784,1472805784,1,2016,9,35,2,4,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN
9,Organic Search,9499785259412240342,9499785259412240342_1472812272,1472812272,1,2016,9,35,2,4,...,NaN,(not set),(not provided),organic,NaN,google,NaN,NaN,NaN,NaN


In [24]:
df, y, nas = proc_df(df_raw, 'transactionRevenue')

In [25]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

0.8631925167374226

In [33]:
print(len(df.index))

903653


In [30]:
def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = 24000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((891653, 78), (891653,), (12000, 78))

In [31]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [32]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

CPU times: user 2min 22s, sys: 384 ms, total: 2min 22s
Wall time: 17.9 s
[0.7370190074690037, 1.742575862762108, 0.8647818933219115, 0.20940112546941259]
